In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from selenium.webdriver.common.by import By
import pandas as pd
from random import randint
from time import sleep
import re
import pickle

In [2]:
def scrape_30_listings(driver):
    # Loop through all 30 entries per page
    for post in range(29):
        
        # Pull job details from the list of jobs on left side of browser window
        listing_str = driver.find_element_by_xpath("(//li[@class='jl selected'])").text
        listing_int = listing_str.split("\n")
        
        # Clean up the data so order of elements is always ['Rating','Job_title','Company','Location', 'Salary_range', 'Status', 'Job_description', 'Company_info'] 
        if not bool(re.search(r'\d+.*', listing_int[0])): 
            listing_int.insert(0, "") # If no star rating available, insert a blank
        listing_lst = listing_int[0:2] + listing_int[2].split(" – ") + listing_int[3:] # Split company and location into 2 fields
        if 'EASY APPLY' in listing_lst: listing_lst.remove('EASY APPLY') # Remove "Easy Apply"
        if len(listing_lst) >= 5:
                if not bool(re.search(r'\D\d+[\w]\s*[-]\D\s*\d+\w+.*', listing_lst[4])): 
                    listing_lst.insert(4, "") # If no estimated salary available, insert a blank
        
        # Pull full text for job description from right side of browser window
        try:
            job_description = driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc'])").text
        except:
            job_description = '-'
        listing_lst.append(job_description)
        
        # Pull bulleted lists from job description from right side of browser window
        try:
            job_description_bullets = driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[1])").text
        except:
            job_description_bullets = '-' 
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[2])").text
        except:
            pass  
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[3])").text
        except:
            pass
        try:
            job_description_bullets = job_description_bullets + " " + driver.find_element_by_xpath("(//div[@class='jobDescriptionContent desc']//ul[3])").text
        except:
            pass
        listing_lst.append(job_description_bullets)
        
        # If the company tab exists for this listing, click on it and scrape company info
        try:
            driver.find_element_by_xpath("(//li[@data-tab-type='overview'])").click()
            sleep(randint(1000,3000)/1000)
            company_str = driver.find_element_by_xpath("(//div[@id='EmpBasicInfo'])").text
        except:
            company_str = '-'
        listing_lst.append(company_str)
        
        # Get the link to the listing
        element = masked_driver.find_element_by_xpath("(//li[@class='jl selected']/div/a)")
        link = element.get_attribute('href')
        listing_lst.append(link)
        
        # Turn the data into a pandas Dataframe and add that df to the list, 'frames'
        listing_df = pd.DataFrame([listing_lst])
        frames.append(listing_df)
        
        # Click on next job posting from the list on left side of browser window
        driver.find_element_by_xpath("(//li[@class='jl'])[position()=" + str(post+1) + "]").click()
        sleep(randint(1000,3000)/1000)
        
    # Stack each entry into one large dataframe    
    features_df = pd.concat(frames)
    return features_df


In [3]:
def go_to_next_page(driver):
    driver.find_element_by_xpath("(//li[@class='next'])").click()
    sleep(randint(3000,5000)/1000)

In [4]:
def collect_mult_pages(masked_driver):
    
    scrape_30_listings(masked_driver)
    
    for i in range(29):
        go_to_next_page(masked_driver)
        sleep(randint(500,1000)/1000)
        x = scrape_30_listings(masked_driver)
        sleep(randint(500,1000)/1000)

    return x

In [5]:
# # Used to test proxy id's

# proxy = '79.134.214.186:32424'
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % proxy)
# masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
# masked_driver.get("https://whatismyipaddress.com/")

In [6]:
proxy_1 = '125.26.7.183:39018'
proxy_2 = '200.122.209.78:47220'
proxy_3 = '217.21.125.179:56774'
proxy_4 = '79.134.214.186:32424'

In [9]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % proxy_2)
masked_driver = webdriver.Chrome(executable_path="/Users/user/Documents/chromedriver2", options=chrome_options)
search_phrase='machine+learning'
masked_driver.get("https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=" + search_phrase + "&sc.keyword=" + search_phrase + "&locT=N&locId=1&fromAge=1&jobType=")


In [10]:
frames=[]
machine_learning = collect_mult_pages(masked_driver)
pickle.dump(machine_learning, open( "machine_learning_posts.pkl", "wb" ) )

In [11]:
machine_learning

,0,1,2,3,4,5,6,7,8,9,10
0,3.8,"Manager, Machine Learning",Amazon,"Sunnyvale, CA",,Today,Job Description\nAlexa is Amazon's new cloud-b...,-,"Overview\nHeadquartersSeattle, WA\nSize10000+ ...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,3.8,Machine Learning Engineer,Xilinx,"San Jose, CA",,Today,Description\n\nXilinx develops highly flexible...,-,"Overview\nHeadquartersSan Jose, CA\nSize1001 t...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,3.7,Principal NLU Researcher/Engineer (Full-Time),DOCOMO Innovations,"Palo Alto, CA",$145k-$202k(Glassdoor Est.),Today,We are seeking an exceptional candidate for a ...,Architect and design efficient data pipelines ...,"Overview\nHeadquartersTokyo, Japan\nSize1 to 5...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,3.7,Data Scientist,Scotts Miracle-Gro,"Columbus, OH",$89k-$132k(Glassdoor Est.),We're Hiring,At Scotts Miracle-Gro we think people are like...,Work entrepreneurially within the organization...,"Overview\nHeadquartersMarysville, OH\nSize5001...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.2,Scientific Data Analyst and Programmer,Stanford University,"Stanford, CA",$63k-$92k(Glassdoor Est.),New,Scientific Data Analyst and Programmer\nStanfo...,Program and optimize pipelines for data qualit...,"Overview\nHeadquartersStanford, CA\nSize10000+...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.6,Data Engineer,Jellyfish,"Reston, VA",$92k-$104k(Glassdoor Est.),Today,"Overview\n\nJellyfish, delivers digital market...","Responsible for all extract, transform and loa...","Overview\nHeadquartersREIGATE, United Kingdom\...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.2,Computer Vision Scientist/Engineer,Baidu USA,"Sunnyvale, CA",$104k-$169k(Glassdoor Est.),Hot,"Baidu USA, located in Silicon Valley, is looki...",Work with world-class talented group of softwa...,"Overview\nHeadquartersSunnyvale, CA\nSize51 to...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,5.0,Woodworker Cabinet Maker,Goebel,"Hutchinson, MN",,We're Hiring,Position: Cabinetmaker/Woodworker\n\nType: Bar...,Able to use common hand tools\nFamiliar with t...,"Overview\nHeadquartersHutchinson, MN\nSize51 t...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,3.6,Java Developer - Machine Learning,Experian,"Scottsdale, AZ",$69k-$138k(Glassdoor Est.),We're Hiring,Experian offers fraud and identity services fo...,A degree in computer science or a similar fiel...,"Overview\nHeadquartersDublin, Ireland\nSize100...",https://www.glassdoor.com/partner/jobListing.h...,NaN
0,4.8,Machine Learning Software Engineer,Decisive Analytics,"Arlington, VA",$44k-$74k(Glassdoor Est.),New,Overview\n\nDECISIVE ANALYTICS Corporation is ...,"Using your creative and technical skills, impl...","Overview\nHeadquartersArlington, VA\nSize51 to...",https://www.glassdoor.com/partner/jobListing.h...,NaN
